In [1]:
import pandas as pd

In [2]:
re1=pd.read_csv('data/reviews_0-250.csv')
re2=pd.read_csv('data/reviews_250-500.csv')
re3=pd.read_csv('data/reviews_500-750.csv')
re4=pd.read_csv('data/reviews_750-1250.csv')
re5=pd.read_csv('data/reviews_1250-end.csv')

C:\Users\Nicolas\AppData\Local\Temp/ipykernel_12432/99614358.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re1=pd.read_csv('data/reviews_0-250.csv')
C:\Users\Nicolas\AppData\Local\Temp/ipykernel_12432/99614358.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re4=pd.read_csv('data/reviews_750-1250.csv')
C:\Users\Nicolas\AppData\Local\Temp/ipykernel_12432/99614358.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re5=pd.read_csv('data/reviews_1250-end.csv')


In [14]:
re5.shape

(49977, 19)

In [3]:
df=pd.concat([re1, re2, re3, re4, re5], ignore_index=True)


In [15]:
df=re5

In [16]:
df=df[['author_id','rating','product_id']]
df.head(10)

,author_id,rating,product_id
0,1945004256,5,P379064
1,5478482359,3,P379064
2,29002209922,5,P379064
3,7391078463,5,P379064
4,1766313888,5,P379064
5,1532951999,5,P379064
6,2168991896,5,P379064
7,5240639417,5,P379064
8,1599604155,5,P379064
9,11792701494,5,P379064


## Dispersión


En los problemas del mundo real, se espera que la matriz de utilidad sea muy dispersa, ya que cada usuario sólo puntua una pequeña fracción de artículos entre el vasto conjunto de opciones disponibles. El problema del arranque en frío puede surgir cuando se añade un nuevo usuario o un nuevo artículo, ya que ambos carecen de historial de valoraciones. La dispersión puede calcularse de la siguietne manera:

In [17]:
#!pip install pyspark
     

In [18]:

from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [19]:
from pyspark.sql import SparkSession

# Detener la sesión de Spark si está activa
try:
    spark.stop()
except:
    pass

# Crear una nueva sesión de Spark con más memoria
spark = SparkSession.builder \
    .appName("ALSExample") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

In [20]:
from pyspark.sql.types import FloatType
df_spark = spark.createDataFrame(df)
df_spark = df_spark.withColumn('rating', df_spark['rating'].cast(FloatType()))
# Count the total number of ratings in the dataset
numerator = df_spark.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = df_spark.select("author_id").distinct().count()
num_movies = df_spark.select("product_id").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

Py4JJavaError: An error occurred while calling o157.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (192.168.0.244 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.

: 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
df_spark = df_spark.withColumn("author_id", df_spark["author_id"].cast(IntegerType()))
df_spark = df_spark.withColumn("rating", df_spark["rating"].cast(IntegerType()))
df_spark = df_spark.withColumn("product_id", df_spark["product_id"].cast(IntegerType()))

In [ ]:
from pyspark.ml.recommendation import ALS

# Configurar el modelo ALS
als = ALS(
    maxIter=10, 
    regParam=0.1, 
    userCol="author_id", 
    itemCol="product_id", 
    ratingCol="rating",
    coldStartStrategy="drop"
)

# Entrenar el modelo ALS
model = als.fit(df_spark)


Py4JJavaError: An error occurred while calling o531.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 4) (192.168.0.244 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.sc

In [ ]:
#pip install implicit


     -------------------------------------- 748.6/748.6 KB 4.7 MB/s eta 0:00:00


You should consider upgrading via the 'c:\Users\Nicolas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
sparse_matrix = csr_matrix((df_spark['rating'], (df_spark['author_id'], df_spark['product_id'])))
model = AlternatingLeastSquares(factors=190, regularization=0.01, iterations=10)
     

PySparkValueError: [CANNOT_CONVERT_COLUMN_INTO_BOOL] Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [ ]:
from scipy.sparse import csr_matrix

# Convertir las columnas necesarias a tipo entero si no lo están aún
df_spark = df_spark.withColumn("author_id", df_spark["author_id"].cast(IntegerType()))
df_spark = df_spark.withColumn("rating", df_spark["rating"].cast(IntegerType()))
df_spark = df_spark.withColumn("product_id", df_spark["product_id"].cast(IntegerType()))

# Crear la matriz dispersa csr_matrix
row_indices = df_spark.select("author_id").rdd.flatMap(lambda x: x).collect()
col_indices = df_spark.select("product_id").rdd.flatMap(lambda x: x).collect()
data_values = df_spark.select("rating").rdd.flatMap(lambda x: x).collect()

sparse_matrix = csr_matrix((data_values, (row_indices, col_indices)))

# Verificar las dimensiones de la matriz dispersa
print("Dimensiones de la matriz dispersa:", sparse_matrix.shape)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 2.0 failed 1 times, most recent failure: Lost task 3.0 in stage 2.0 (TID 8) (192.168.0.244 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apac